In [1]:
import torch #main torch module
import torch.nn as nn #neural net module
import torch.optim as optim #optimizers
import torch.nn.functional as F #functions like ReLu Sig Tanh etc
from torch.utils.data import DataLoader #help us with datasets

import torchvision.datasets as datasets #using to access std data
import torchvision.transforms as transforms #transformations

### Network Definition
We will create BI LSTM

In [3]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate =  0.001
batch_size = 64
num_epochs = 2

Now we will define BRNN

In [4]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm= nn.LSTM(input_size, hidden_size, num_layers,
                          batch_first = True, bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
    
    def forward(self,x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        out, (hidden_state, cell_state) = self.lstm(x, (h0,c0))
        out = self.fc(out[:,-1,:])
        return out

### Modelling and Evaluation

In [5]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Loading Data
train_dataset = datasets.MNIST(root='dataset/',
                               train=True, 
                               transform = transforms.ToTensor(),
                               download = True)

train_loader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [7]:
# Loading Test Data
test_dataset = datasets.MNIST(root='dataset/',
                               train=False, 
                               transform = transforms.ToTensor(),
                               download = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [8]:
#init the network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [9]:
# training loop
for epoch in range(num_epochs):
    for batch_idx,(data, targets) in enumerate(train_loader):
        # data to devices
        data = data.to(device).squeeze(1) #squeeze 64x1x28x28 to 64x28x28
        targets = targets.to(device)
        
        #fwd
        scores = model(data)
        loss = criterion(scores, targets)
        
        #back
        optimizer.zero_grad() #so that it does not store prev backprop calc
        loss.backward()
        
        #gradient desc
        optimizer.step()

In [10]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("Checking Training Data Accuracy")
    else:
        print("Checking Test Data Accuract")
    
    num_correct = 0
    num_samples = 0
    model.eval() #set to evaluation mode
    
    with torch.no_grad():
        #only have to check accuracy, dont compute grads
        for x,y in loader:
            x = x.to(device).squeeze(1)
            y = y.to(device)
            #x = x.reshape(x.shape[0],-1)
            scores = model(x)
            _, predictions = scores.max(1)
            
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        accuracy = float(num_correct)/float(num_samples)*100
        print(f"Got {num_correct} / {num_samples} with accuracy {accuracy: .2f}")

    model.train()
    return accuracy

In [11]:
check_accuracy(test_loader,model)
check_accuracy(train_loader,model)

Checking Test Data Accuract
Got 9761 / 10000 with accuracy  97.61
Checking Training Data Accuracy
Got 58634 / 60000 with accuracy  97.72


97.72333333333333